In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 



Entrenamiento provedido en Español <br>
Training offered in English

<img src="https://www.ciesin.columbia.edu/images/logo_sm.png" >

# ANALYSIS DE PUNTOS CRITICOS DE VULNERABILIDAD - AmeriGEO 2023 San Jose
# HOTSPOT VULNERABILITY ANALYSIS - AmeriGEO 2023 San Jose

### JUAN F. MARTINEZ, Center for International Earth Science Information Network (CIESIN)

#### The Hotspot vulnerability training will provide a comprehensive training in the framework, data, and methods used to develop a spatial vulnerability assessment.
##### The training will utilize open-acccess software and data to demonstrate how to develop a final hotspot analysis product using different sources and formats of data. The training will take three stages:
##### 1. Theoretical Framework of the Index Approach to Hotspospot Vulnerability Mapping 
##### 2. Processing of Different Data Products: Case Study in Municipalities of Cauca, Colombia
##### 3. Indexing and Aggregating Process

## 1. Theoretical Framework of the Index Approach
The United Nation's 2030 Sustainable Development Agenda has provided us a blueprint for peace and prosperity for people and the planet, now and into the future. <br>
The 17 Sustainable Development Goals (SDG's) are the main pillars of the Agenda that call for global partnership from all member countries. <br>
<img src="https://www.undp.org/sites/g/files/zskgke326/files/migration/mm/sdg-sticker-eng.jpg" width="50%"><br>
##### The first Sustainable Development Goal (SDG) in the United Nations (UN) 2030 Agenda seeks to end poverty of all forms everywhere by reaching the poorest of the poor, and to ensure progress for all population groups. <br>
##### Poverty has been traditionally measured by individual economic dimensions of income and wealth. For example, the <a href="https://www.worldbank.org/en/understanding-poverty#:~:text=Regions%20are%20categorized%20using%20PIP%20definition.&text=Note%20on%20global%20poverty%20lines,%242.15%20per%20person%20per%20day." target="_blank">World Bank defined the global extreme-poverty line as $2.15 USD (2022)</a> income or below per day, which is based on the national poverty lines of the world’s 15 poorest countries. However, these indicators do not entirely consider other non-financial forms of poverty that may be of equal importance. The concept of multidimensional poverty encompasses a more holistic view because it attempts to account for the lived experiences of people and the multiple deprivations they face in their daily lives beyond their incomes.

## Examples of Multidimensional Poverty Indices


  


### Data Sources:
<a href="https://geoportal.dane.gov.co/servicios/descarga-y-metadatos/descarga-mgn-marco-geoestadistico-nacional/#gsc.tab=0">Colombia Municipal Boundaries</a>

<a href="https://geoportal.dane.gov.co/geovisores/sociedad/indicadores-regionales/"> Valle del Cauca Proportion of people in poverty</a>

In [ ]:
#install libraries required for this tutorial
pip install geopandas rasterio matplotlib numpy pandas rasterstats


In [ ]:
#import libraries required for this tutorial
import geopandas as gpd
import rasterio
from rasterio import mask
from rasterio.plot import show
from rasterio.plot import show_hist
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rasterstats import zonal_stats



In [ ]:
#file such as a geoTiff or shapefile used to delineate zones. 
#Values shoud be integers
zones = gpd.read_file("data/Col_CAUCA_municip.shp")

#files of data used to perform zonal statistics

#Land Surface Temperature (LST)
LST = rasterio.open("data/summer_LST.tif")
#freshwater availability (water)
water = rasterio.open("data/freshwater.tif")
#ozone particulate matter 2.5 (PM25)
PM25 = rasterio.open("data/PM25.tif")


#read LST raster band 1 
LST_data = LST.read(1, masked=True)
#mask null value
#LST_data = np.ma.masked_array(LST_r, mask=(LST_r < 0))

#read LST raster band 1
water_data = water.read(1, masked=True)
#water_data = np.ma.masked_array(water)

#read LST raster band 1
PM25_data = PM25.read(1, masked=True)
#mask null value
#PM25_data = np.ma.masked_array(PM25_r, mask=(PM25_r < 0))



In [ ]:
#create a plot with 1 row and 4 columns limiting size to 20 by 5
fig, ax = plt.subplots(1,4, figsize = (20,5))
#plot the zones shapefile in the first column named ax[0] 
zones.plot(cmap = 'rainbow', ax=ax[0])

#plot rasters in the rest of the columns
im1 = ax[1].imshow(LST_data, cmap='plasma')
im2 = ax[2].imshow(water_data, cmap='GnBu')
im3 = ax[3].imshow(PM25_data, cmap='viridis')

#set titles for graphs
ax[0].set_title('Cauca - Municipalidades', wrap=True)
ax[1].set_title('Cauca - Summer Land\nSurface Temperature', wrap=True);
ax[2].set_title('Cauca - Freshwater Availability', wrap=True);
ax[3].set_title('Cauca - Particulate Matter 2.5 (PM2.5)', wrap=True);

#color bar for rasters
fig.colorbar(im1, ax=ax[1])
fig.colorbar(im2, ax=ax[2])
fig.colorbar(im3, ax=ax[3])

plt.show()

In [ ]:
rasterio.plot.show_hist(LST_data)
rasterio.plot.show_hist(water_data)
rasterio.plot.show_hist(PM25_data)



In [ ]:

#loop through zone IDs
for i in zones['MID'][:1]:
    print(i)
    #get single shape from shapefile
    roi = zones[zones.MID == i]
    LST_arr, LSTbound = mask.mask(LST, roi["geometry"], crop=True, all_touched=True, nodata=-9999)
    water_arr, waterbound = mask.mask(water, roi["geometry"], crop=True, all_touched=True, nodata=-9999)
    PM25_arr, PM25bound = mask.mask(PM25, roi["geometry"], crop=True, all_touched=True, nodata=-9999) 
    LST_arr[LST_arr ==-9999] = None
    water_arr[water_arr ==-9999] = None
    PM25_arr[PM25_arr ==-9999] = None
show(LST_arr)  
show(water_arr) 
show(PM25_arr)    

In [ ]:

df = pd.DataFrame(columns =['MID','LST','water','PM25'])

#loop through zone IDs
for i in zones['MID']:
    #print(i)
    #get single shape from shapefile
    roi = zones[zones.MID == i]
    LST_arr, LSTbound = mask.mask(LST, roi["geometry"], crop=True, all_touched=True, nodata=-9999)
    water_arr, waterbound = mask.mask(water, roi["geometry"], crop=True, all_touched=True, nodata=-9999)
    PM25_arr, PM25bound = mask.mask(PM25, roi["geometry"], crop=True, all_touched=True, nodata=-9999) 
    LST_arr[LST_arr ==-9999] = None
    water_arr[water_arr ==-9999] = None
    PM25_arr[PM25_arr ==-9999] = None
    row_to_append = pd.DataFrame([{'MID':i, 'LST': np.nanmean(LST_arr), 
                                   'water':np.nanmean(water_arr),
                                   'PM25': np.nanmean(PM25_arr)}])
    df = pd.concat([df,row_to_append])
df

 

In [ ]:
df['LST_index'] = (df['LST']-df['LST'].min())/(df['LST'].max()- df['LST'].min())*100
df['water_index'] = (df['water']-df['water'].max())/(df['water'].min()- df['water'].max())*100
df['PM25_index'] = (df['PM25']-df['PM25'].min())/(df['PM25'].max()- df['PM25'].min())*100
df['HVI_sum']= df['LST_index']+ df['water_index']+df['PM25_index']
df['HVI'] = (df['HVI_sum']-df['HVI_sum'].min())/(df['HVI_sum'].max()- df['HVI_sum'].min())*100
df

In [ ]:
zones = zones.merge(df, on='MID')

In [ ]:
zones.plot(column='HVI', cmap='inferno', legend=True)